In [27]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 49.26,
	"longitude": -123.25,
	"daily": ["temperature_2m_max", "temperature_2m_min", "sunrise"],
	"hourly": ["temperature_2m", "apparent_temperature", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth"],
	"current": ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "precipitation", "rain", "showers", "snowfall", "cloud_cover", "wind_speed_10m"],
	"minutely_15": ["precipitation", "temperature_2m", "apparent_temperature", "rain", "snowfall", "snowfall_height", "wind_gusts_10m", "wind_speed_10m"],
	"timezone": "America/Los_Angeles",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process current data. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_relative_humidity_2m = current.Variables(1).Value()
current_apparent_temperature = current.Variables(2).Value()
current_precipitation = current.Variables(3).Value()
current_rain = current.Variables(4).Value()
current_showers = current.Variables(5).Value()
current_snowfall = current.Variables(6).Value()
current_cloud_cover = current.Variables(7).Value()
current_wind_speed_10m = current.Variables(8).Value()

print(f"\nCurrent time: {current.Time()}")
print(f"Current temperature_2m: {current_temperature_2m}")
print(f"Current relative_humidity_2m: {current_relative_humidity_2m}")
print(f"Current apparent_temperature: {current_apparent_temperature}")
print(f"Current precipitation: {current_precipitation}")
print(f"Current rain: {current_rain}")
print(f"Current showers: {current_showers}")
print(f"Current snowfall: {current_snowfall}")
print(f"Current cloud_cover: {current_cloud_cover}")
print(f"Current wind_speed_10m: {current_wind_speed_10m}")

# Process minutely_15 data. The order of variables needs to be the same as requested.
minutely_15 = response.Minutely15()
minutely_15_precipitation = minutely_15.Variables(0).ValuesAsNumpy()
minutely_15_temperature_2m = minutely_15.Variables(1).ValuesAsNumpy()
minutely_15_apparent_temperature = minutely_15.Variables(2).ValuesAsNumpy()
minutely_15_rain = minutely_15.Variables(3).ValuesAsNumpy()
minutely_15_snowfall = minutely_15.Variables(4).ValuesAsNumpy()
minutely_15_snowfall_height = minutely_15.Variables(5).ValuesAsNumpy()
minutely_15_wind_gusts_10m = minutely_15.Variables(6).ValuesAsNumpy()
minutely_15_wind_speed_10m = minutely_15.Variables(7).ValuesAsNumpy()

minutely_15_data = {"date": pd.date_range(
	start = pd.to_datetime(minutely_15.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(minutely_15.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = minutely_15.Interval()),
	inclusive = "left"
)}

minutely_15_data["precipitation"] = minutely_15_precipitation
minutely_15_data["temperature_2m"] = minutely_15_temperature_2m
minutely_15_data["apparent_temperature"] = minutely_15_apparent_temperature
minutely_15_data["rain"] = minutely_15_rain
minutely_15_data["snowfall"] = minutely_15_snowfall
minutely_15_data["snowfall_height"] = minutely_15_snowfall_height
minutely_15_data["wind_gusts_10m"] = minutely_15_wind_gusts_10m
minutely_15_data["wind_speed_10m"] = minutely_15_wind_speed_10m

minutely_15_dataframe = pd.DataFrame(data = minutely_15_data)
print("\nMinutely15 data\n", minutely_15_dataframe)

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_showers = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
# hourly_data["snow_depth"] = hourly_snow_depth

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_sunrise = daily.Variables(2).ValuesInt64AsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["sunrise"] = daily_sunrise

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)


Coordinates: 49.24641799926758°N -123.23323822021484°E
Elevation: 98.0 m asl
Timezone: b'America/Los_Angeles'b'GMT-8'
Timezone difference to GMT+0: -28800s

Current time: 1763953200
Current temperature_2m: 5.741999626159668
Current relative_humidity_2m: 82.0
Current apparent_temperature: 1.7663359642028809
Current precipitation: 0.0
Current rain: 0.0
Current showers: 0.0
Current snowfall: 0.0
Current cloud_cover: 100.0
Current wind_speed_10m: 16.071491241455078

Minutely15 data
                          date  precipitation  temperature_2m  \
0   2025-11-23 08:00:00+00:00            0.0           8.142   
1   2025-11-23 08:15:00+00:00            0.4           8.142   
2   2025-11-23 08:30:00+00:00            0.2           8.242   
3   2025-11-23 08:45:00+00:00            0.1           8.242   
4   2025-11-23 09:00:00+00:00            0.0           8.092   
..                        ...            ...             ...   
283 2025-11-26 06:45:00+00:00            0.0           5.900   
284 